In [1]:
'''! pip install lightning
! pip install torchmetrics
! pip install watermark
! pip install mlxtend
! pip install pandas'''

'! pip install lightning\n! pip install torchmetrics\n! pip install watermark\n! pip install mlxtend\n! pip install tensorboard\n! pip install pandas'

In [1]:
import time

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms

import torchmetrics
import timm
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datamodules import RSNAdataset
from utils import LossMeter
from plotting import show_failures, plot_loss_and_acc

/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb
wandb.login(key='a2a7828ed68b3cba08f2703971162138c680b664')

wandb: Currently logged in as: karanjot. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/karanjotvendal/.netrc


True

In [4]:
%reload_ext watermark
%watermark -a 'Karanjot Vendal' -v -p torch --iversion

Author: Karanjot Vendal

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

torch: 2.0.1

torchvision : 0.15.2
numpy       : 1.25.1
torch       : 2.0.1
timm        : 0.9.2
matplotlib  : 3.7.2
wandb       : 0.15.7
torchmetrics: 1.0.1
pandas      : 2.0.3



# Hyperparameters

In [8]:
PATH = 'data/reduced_dataset/'
MODEL = 'resnet18'
BATCH_SIZE = 3
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001

NUM_WORKERS = 2
NUM_CLASSES = 2

KFOLD= 10 
# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
run = wandb.init(
      # Set the project where this run will be logged
      project="RACNet", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{1}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": LEARNING_RATE,
      "architecture": "CNN-GRU-MASK-FC-Classifier",
      "dataset": "MICAA MRI",
      "epochs": NUM_EPOCHS,
      "BATCH_SIZE": BATCH_SIZE
      })

# Model

In [9]:
class RACNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.cnn = timm.create_model(MODEL, pretrained=True, num_classes=0, in_chans=1)
        for param in self.cnn.parameters():
            param.requires_grad = False
        in_features = self.cnn(torch.randn(2, 1, 112, 112)).shape[1]
        
        self.rnn = nn.GRU(input_size=in_features, hidden_size=64, batch_first= True, bidirectional=False)
        
        self.fc = nn.Linear(16256, 32, bias=True)
        self.classifier = nn.Linear(32, num_classes, bias=True)

    def forward(self, x, org):
        # x shape: BxSxCxHxW
        batch_size, slices, C, H, W = x.size()
        c_in = x.view(batch_size * slices, C, H, W)
        print('reshape input', c_in.shape)
        
        out = self.cnn(c_in)
        print('CNN ouput', out.shape)
        
        rnn_in = out.view(batch_size, slices, -1)
        print('reshaped rnn_in', rnn_in.shape)
        out, hd = self.rnn(rnn_in)
        #out =F.relu(self.RNN(out))

        print('RNN ouput', out.shape)
        mask = self.mask_layer(org)
        out = out * mask
        print('mask ouput', out.shape)
        
        batch, slices, rnn_features = out.size()
        out = out.reshape(batch_size, slices * rnn_features)
        print('reshaped masked output', out.shape)
        
        out = F.relu(self.fc(out))
        print('fc ouput', out.shape)
        
        logits = self.classifier(out)
        print('logits', logits.shape)
        
        output = F.softmax(logits, dim=1)
        
        print('classifier ouput', logits.shape)
        #[prob 0, prob 1]

        return logits, output

    def mask_layer(self, org):
        masks = []
        org = org[0].cpu().numpy()
        for i in org:
            #print(i)
            dup = 254 - i
            mask_1 = torch.ones(i, 64) # .to(device='cuda')
            mask_0 = torch.zeros(dup, 64) #.to(device='cuda')
            mask = torch.cat((mask_1, mask_0), 0)
            masks.append(mask)

        masks = torch.stack(masks).to(device='cuda')
        return masks

In [4]:
for batch in dm.train_dataloader():
    #train_counter.update(labels.tolist())
    #print('X',batch['X'])
    #print('y',batch['y'])
    print('org',batch['org'][0][1])
    #for i in batch['org']:
    #    print(i.items())
    break
    

NameError: name 'dm' is not defined

# Training

In [10]:
class Trainer():
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion,
        epochs,
        loss_meter, 
        fold
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.loss_meter = loss_meter
        self.hist = {'test_acc':[],
                     'test_f1':[],
                     'test_roc':[],
                     'train_loss':[],
                     'train_acc':[],
                     'train_f1': [],
                     'train_roc': [],
                    }
        
        self.best_valid_score = -np.inf
        self.best_valid_loss = np.inf
        self.best_f_score = 0
        self.n_patience = 0
        self.fold = fold

        self.record = {'test_loss':[],
                    'test_acc':[],
                     'test_f1':[],
                     'test_roc':[],
                     'train_loss':[],
                     'train_acc':[],
                     'train_f1': [],
                     'train_roc': [],
                    }
        
        
    def fit(self, epochs, train_loader, save_path, patience):
        train_time = time.time()
        
        for epoch in range(epochs):
            t = time.time()
            self.model.train()
            train_loss = self.loss_meter()
            train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2).to(self.device)
            train_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2, average='macro').to(self.device)
            train_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2).to(self.device)
            
            for idx, batch in enumerate(tqdm(train_loader)):
                print('-'*50)
                features = batch['X'].to(self.device)
                targets = batch['y'].type(torch.cuda.LongTensor).to(self.device)
                org = batch['org']
                print(org)
                print("feature", features.shape)
                ### FORWARD AND BACK PROP
                logits, probs = self.model(features, org)
                loss = self.criterion(logits, targets)
                
                train_loss.update(loss.detach().item())
                train_acc.update(probs, targets)
                train_f1.update(probs, targets)
                train_auroc.update(probs, targets)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()  
                print(f'-----------------Loss: {loss}-----------------------')
                print('------BATCH ENDING-------')
            
                #print(f'Epoch: {epoch+1}/{epochs} | Loss: {loss:.5f} | Accuracy: {train_acc:.4f}% | F1 Score: {train_f1:.4f} | AUROC: {train_roc:.4f} | Time: {int(time.time() - t)}')
            
            _loss = train_loss.avg
            _acc = train_acc.compute()
            _f1 = train_f1.compute()
            _roc = train_auroc.compute()
            
            self.hist['train_loss'].append(_loss)
            self.hist['train_acc'].append(_acc)
            self.hist['train_f1'].append(_f1)
            self.hist['train_auroc'].append(_roc)
            
            print(f'Epoch: {epoch+1}/{epochs} | Loss: {_loss:.5f} | Accuracy: {_acc:.4f}% | F1 Score: {_f1:.4f} | AUROC: {_roc:.4f} | Time: {time.time() - t}')
            
            
        avg_loss = np.mean(self.hist['train_loss'])
        avg_acc = np.mean(self.hist['train_acc'])
        avg_f1 = np.mean(self.hist['train_f1'])
        avg_auroc = np.mean(self.hist['train_auroc'])

        print(f'Epoch Training Time: {(train.time() - start_time)/60} min | Avg Loss: {avg_loss:.5f} | Avg Accuracy: {avg_acc:.4f}% | Avg F1 Score: {avg_f1:.4f} | Avg AUROC: {avg_auroc:.4f}')
        
        
        
        #return avg_loss, avg_acc, avg_f1, avg_auroc
        
        
        #testing------------------
    
    def test(self, test_loader):
        test_time = time.time()
        test_loss = self.loss_meter()
        test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        test_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2, average='macro')        
        test_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)
        self.model.eval()
        for idx, batch in enumerate(test_loader):
            with torch.no_grad():    
                features = batch['X'].to(self.device)
                #targets = batch['y'].type(torch.cuda.LongTensor).to(self.device)
                targets = batch['y'].type(torch.cuda.LongTensor).to(self.device)
                
                org = batch['org']
                print(org)
                
                logits, probas = self.model(features, org)
                loss = self.criterion(logits, targets)
            
                test_loss.update(loss.detach().item())
                test_acc.update(probs, targets)
                test_f1.update(probs, targets)
                test_auroc.update(probs, targets)
                print('------BATCH ENDING-------')

        _loss = train_loss.avg
        _acc = train_acc.compute()
        _f1 = train_f1.compute()
        _roc = train_auroc.compute()  

        self.hist['test_loss'].append(_loss)
        self.hist['test_acc'].append(_acc)
        self.hist['test_f1'].append(_f1)
        self.hist['test_auroc'].append(_roc)
            

                #print(f'Total Training Time: {(train.time() - start_time)/60} min | Avg Loss: {avg_loss:.5f} | Avg Accuracy: {avg_acc:.4f}% | Avg F1 Score: {avg_f1:.4f} | Avg AUROC: {avg_auroc:.4f}')


        avg_loss = np.mean(self.hist['test_loss'])
        avg_acc = np.mean(self.hist['test_acc'])
        avg_f1 = np.mean(self.hist['test_f1'])
        avg_auroc = np.mean(self.hist['test_auroc'])

        print(f'Testing Time: {(time.time() - test_time)/60} min | Avg Loss: {avg_loss:.5f} | Avg Accuracy: {avg_acc:.4f}% | Avg F1 Score: {avg_f1:.4f} | Avg AUROC: {avg_auroc:.4f}')
        
        return avg_loss, avg_acc, avg_f1, avg_auroc
                        
                
    def save_model(self, n_epoch, save_path):
            torch.save(
                {
                    "model_state_dict": self.model.state_dict(),
                    "optimizer_state_dict": self.optimizer.state_dict(),
                    "best_valid_score": self.best_valid_score,
                    "best_f1_score": self.best_f_score,
                    "n_epoch": n_epoch,
                },
                save_path,
            )

In [11]:
def train(path, epochs, n_fold, batch_size, num_workers, device):
    
    fold_acc = []
    fold_loss = []
    fold_auroc = []
    fold_f1 = []

    start_time = time.time()
    model = RACNet(NUM_CLASSES)
    model = model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = F.cross_entropy
        
    for _ in range(n_fold):
        fold = _+1
        folds_xtrain = np.load('./data/folds/xtrain.npy', allow_pickle=True)
        folds_xtest = np.load('./data/folds/xtest.npy', allow_pickle=True)
        folds_ytrain = np.load('./data/folds/ytrain.npy', allow_pickle=True)
        folds_ytest = np.load('./data/folds/ytest.npy', allow_pickle=True)
        
        xtrain = folds_xtrain[_]
        ytrain = folds_ytrain[_]
        xtest = folds_xtest[_]
        ytest = folds_ytest[_]
        
        print('-'*30)
        print(f"Fold {fold}")

        dlt = []
        for idx, value in enumerate(xtrain):
            if value == 109:
                dlt.append(idx)
            if value ==123:
                dlt.append(idx)
            if value == 709:
                dlt.append(idx)
                
        for i in dlt:
            np.delete(xtrain, i)
            np.delete(ytrain, i)

        d = []
        for idx, value in enumerate(xtest):
            if value == 109 or value ==123 or value == 709:
                d.append(idx)
        for i in d:
            np.delete(xtest, i)
            np.delete(ytest, i)
                

        train_set = RSNAdataset(
                        './data/reduced_dataset/',
                        xtrain,  
                        ytrain,
                        n_slices=254,
                        img_size=112,
                        transform=None
                            )
    
        test_set = RSNAdataset(
                        './data/reduced_dataset/',
                        xtest,  
                        ytest,
                        n_slices=254,
                        img_size=112,
                        transform=None
                            )
        
        train_loader = DataLoader(
                    train_set,    
                    batch_size=batch_size,
                    shuffle=True,
                    num_workers=num_workers,
                )
        
        test_loader = DataLoader(
                    test_set,    
                    batch_size=batch_size,
                    shuffle=True,
                    num_workers=num_workers,
                )
        #for batch in train_loader:
        #    print('X',batch['X'])
        #    print('y',batch['y'])
        #    print('org',batch['org'])
            #for i in batch['org']:
            #    print(i.items())
         #   break
                    
        #model = RACNet(NUM_CLASSES)
        #model = model.to(DEVICE)
        #optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
        #criterion = F.cross_entropy
        
        trainer = Trainer(
            model, 
            device, 
            optimizer, 
            criterion,
            epochs,
            LossMeter, 
            fold
        )
        
        trainer.fit(epochs,
                    train_loader,
                    './checkpoints/f"best-model-{fold}.pth',
                    5)
                        
        #trainer.plot_loss()
        #trainer.plot_score()
        #trainer.plot_fscore()
                
        #test
        loss, test_acc, test_f1, test_auroc = Trainer.test(test_loader)
        fold_loss.append(loss)
        fold_acc.append(test_acc)
        fold_f1.append(test_f1)
        fold_auroc.append(test_auroc)    
    
    elapsed_time = time.time() - start_time
    '''wandb.log({
         'Avg Test f1 score': np.mean(test_fscore),
         'Avg Train f1 score': np.mean(f_scores)
         })'''
    print('\nTraining complete in {:.0f}m {:.0f}s'.format(elapsed_time // 60, elapsed_time % 60))
    print('Avg loss {:.5f}'.format(np.mean(losses)))
    print('Avg score {:.5f}'.format(np.mean(scores)))
    print('Avg Train f1_score {:.5f}'.format(np.mean(f_scores)))
    print('Avg Test f1_score {:.5f}'.format(np.mean(test_fscore)))

In [12]:
train(PATH, NUM_EPOCHS, KFOLD, BATCH_SIZE, NUM_WORKERS, DEVICE)

------------------------------
Fold 1


  0%|                                                                                           | 0/156 [00:00<?, ?it/s]

--------------------------------------------------
[tensor([ 96, 101,  31])]
feature torch.Size([3, 254, 1, 112, 112])
reshape input torch.Size([762, 1, 112, 112])
CNN ouput torch.Size([762, 512])
reshaped rnn_in torch.Size([3, 254, 512])
RNN ouput torch.Size([3, 254, 64])


  1%|█                                                                                  | 2/156 [00:01<01:13,  2.11it/s]

mask ouput torch.Size([3, 254, 64])
reshaped masked output torch.Size([3, 16256])
fc ouput torch.Size([3, 32])
logits torch.Size([3, 2])
classifier ouput torch.Size([3, 2])
-----------------Loss: 0.7153329253196716-----------------------
------BATCH ENDING-------
--------------------------------------------------
[tensor([ 93,  34, 102])]
feature torch.Size([3, 254, 1, 112, 112])
reshape input torch.Size([762, 1, 112, 112])
CNN ouput torch.Size([762, 512])
reshaped rnn_in torch.Size([3, 254, 512])
RNN ouput torch.Size([3, 254, 64])
mask ouput torch.Size([3, 254, 64])
reshaped masked output torch.Size([3, 16256])
fc ouput torch.Size([3, 32])
logits torch.Size([3, 2])
classifier ouput torch.Size([3, 2])
-----------------Loss: 0.7232459187507629-----------------------
------BATCH ENDING-------
--------------------------------------------------
[tensor([111,  20,  46])]
feature torch.Size([3, 254, 1, 112, 112])
reshape input torch.Size([762, 1, 112, 112])


...

  3%|██▋                                                                                | 5/156 [00:01<00:34,  4.32it/s]

mask ouput torch.Size([3, 254, 64])
reshaped masked output torch.Size([3, 16256])
fc ouput torch.Size([3, 32])
logits torch.Size([3, 2])
classifier ouput torch.Size([3, 2])
-----------------Loss: 0.6910204291343689-----------------------
------BATCH ENDING-------
--------------------------------------------------
[tensor([19, 92, 49])]
feature torch.Size([3, 254, 1, 112, 112])
reshape input torch.Size([762, 1, 112, 112])
CNN ouput torch.Size([762, 512])
reshaped rnn_in torch.Size([3, 254, 512])
RNN ouput torch.Size([3, 254, 64])
mask ouput torch.Size([3, 254, 64])
reshaped masked output torch.Size([3, 16256])
fc ouput torch.Size([3, 32])
logits torch.Size([3, 2])
classifier ouput torch.Size([3, 2])
-----------------Loss: 0.7030413746833801-----------------------
------BATCH ENDING-------
--------------------------------------------------
[tensor([ 97, 112,  32])]
feature torch.Size([3, 254, 1, 112, 112])
reshape input torch.Size([762, 1, 112, 112])
CNN ouput torch.Size([762, 512])
resh

  4%|███▏                                                                               | 6/156 [00:01<00:44,  3.41it/s]


mask ouput torch.Size([3, 254, 64])
reshaped masked output torch.Size([3, 16256])
fc ouput torch.Size([3, 32])
logits torch.Size([3, 2])
classifier ouput torch.Size([3, 2])
-----------------Loss: 0.6867237091064453-----------------------
------BATCH ENDING-------


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 127, in collate
    return elem_type({key: collate([d[key] for d in batch], collate

# Evaluating the model

In [ ]:
#visualising cofusion matrix
from torchmetrics import ConfusionMatrix
import matplotlib
from mlxtend.plotting import plot_confusion_matrix

cmat = ConfusionMatrix(task='multiclass', num_classes=NUM_CLASSES)

for x, y in dm.test_dataloader():

    with torch.no_grad():
        pred = lightning_model(x)
    cmat(pred, y)

cmat_tensor = cmat.compute()
cmat = cmat_tensor.numpy()

fig, ax = plot_confusion_matrix(
    conf_mat=cmat,
    class_names=class_dict.values(),
    norm_colormap=matplotlib.colors.LogNorm()  
    # normed colormaps highlight the off-diagonals 
    # for high-accuracy models better
)

plt.show()

In [26]:
import cv2

In [27]:
def load_image(path, size=(112,112)):
    image = cv2.imread(path, 0)
    if image is None:
        return np.zeros()
    
    image = cv2.resize(image, size) / 255
    return image.astype('f')

In [28]:
paths =[]
images=[load_image(path) for path in paths]
org_size = len(images)

In [31]:
dup_len = 254 - len(images)
if org_size == 0:
    dup = torch.zeros(1, 112, 112)
    print('dup', dup)
else:
    dup = images[-1]
for i in range(dup_len):
    images.append(dup)

images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.stack(images)



dup tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


/tmp/ipykernel_3239/2694937284.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image, dtype=torch.float32) for image in images]


In [32]:
images.shape

torch.Size([254, 1, 112, 112])

In [16]:
img = load_image('data/reduced_dataset/00002/FLAIR/Image-437.png')

In [17]:
img.shape

(112, 112)

In [19]:
from PIL import Image

In [23]:
ig = plt.imread('data/reduced_dataset/00002/FLAIR/Image-437.png')

In [25]:
ig

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
image = []
image.append(img)
image

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]

In [19]:
dup = image[-1]
dup


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
for i in range(4):
    image.append(dup)

image

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],


In [25]:
image = [torch.tensor(image, dtype=torch.float32) for image in image]


In [26]:
image

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

In [27]:
image[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [29]:
image[0].shape


torch.Size([112, 112])

In [30]:
image = torch.stack(image)

In [31]:
image.shape

torch.Size([5, 112, 112])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [35]:
images = []
img = load_image('data/reduced_dataset/00002/FLAIR/Image-437.png')
images.append(img)

dup = images[-1]

for i in range(4):
    images.append(dup)

images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.vstack(images)

In [37]:
images

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [4]:
org = [0]
masks= []
for i in org:
    #print(i)
    dup = 254 - i
    mask_1 = torch.ones(i, 64) # .to(device='cuda')
    mask_0 = torch.zeros(dup, 64) #.to(device='cuda')
    mask = torch.cat((mask_1, mask_0), 0)
    masks.append(mask)

masks = torch.stack(masks)
mask.shape

torch.Size([254, 64])

In [3]:
n = np.zeros((1,112,112))

In [2]:
images = []
org_size = 150
dup_len = 254 - org_size
dup = np.zeros((1,112,112))
for i in range(150):
    images.append(dup)

dup = images[-1]
for i in range(dup_len):
    images.append(dup)

images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.stack(images)

In [3]:
images.shape

torch.Size([254, 1, 112, 112])

In [4]:
images = []
dup = np.zeros((1,112,112))  #torch.zeros(1, 112, 112).numpy()

for i in range(254):
    images.append(dup)

In [11]:
len(images)

#[tensor([0])]
#feature shape torch.Size([1, 254, 1, 1, 112, 112])

254

In [5]:
images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.stack(images)
images.shape

torch.Size([254, 1, 112, 112])

In [4]:
data =[]
data.append(images)
data = torch.stack(data)
data.shape

torch.Size([1, 254, 1, 112, 112])

In [9]:
data.transpose(1,0).shape

torch.Size([254, 1, 1, 112, 112])

In [10]:
d =[]
d.append(images)
d = torch.stack(d)
d.shape

torch.Size([1, 254, 1, 112, 112])

In [11]:
d.transpose(1,0).shape

torch.Size([254, 1, 1, 112, 112])